## Part X.2 - Configure CI/CD Pipeline for DeepAR 

University of San Diego - MS Applied AI

AAI-540 Team 5

October 21, 2024

In [1]:
# setup environment
%run 0-Environment_Setup.ipynb

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Stored 's3_datalake_path_csv' (str)
Stored 'local_data_path_csv' (str)
Stored 's3_datalake_path_parquet' (str)


In [27]:
# import necessary libraries
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

In [9]:
# init pipeline session
pipeline_session = PipelineSession()

In [3]:
# Setup S3 buckets
input_data_path = "s3://{}/store-sales-forecasting/deepar/pipelines/train/input".format(bucket)
batch_data_path = "s3://{}/store-sales-forecasting/deepar/pipelines/train/output".format(bucket)

In [73]:
# get latest feature store and write local
input_local_file = './test-data/input_data.csv'
sales_features_store = get_store_dataset_from_offline_feature_group(store_sales_feature_group)
sales_features_store.to_csv(input_local_file)

# set destination path in S3
input_data_file = "{}/input_data.csv".format(input_data_path)

INFO:sagemaker:Query 91623cbb-4497-482f-9750-ee51e862d6ae is being executed.


Running 
    SELECT *
    FROM
        "store_sales_feature_group_offline_1728336748"
    ORDER BY
        store_nbr ASC, date ASC
    


INFO:sagemaker:Query 91623cbb-4497-482f-9750-ee51e862d6ae successfully executed.


In [74]:
# copy validation dataset local
!aws s3 cp $input_local_file $input_data_file

upload: test-data/input_data.csv to s3://sagemaker-us-east-1-343218227212/store-sales-forecasting/deepar/pipelines/train/input/input_data.csv


In [75]:
# Load the best model information from our tuning job
tuning_job_name = "deepar-hyperparamete-241007-2220"
tuning_job_result = sm.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

# get model details from best training job
best_training_job_name = tuning_job_result["BestTrainingJob"]["TrainingJobName"]
best_training_job = sm.describe_training_job(TrainingJobName=best_training_job_name)

# get the best RMSE score to use in pipeline
best_rmse_metric = 0
for metric_name in best_training_job['FinalMetricDataList']:
    if(metric_name['MetricName'] == 'test:RMSE'):
        best_rmse_metric = metric_name['Value']

print("Best model test:RMSE score: {}".format(best_rmse_metric))

Best model test:RMSE score: 3434.864990234375


In [77]:
# define pipeline parameters
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.2xlarge")
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)
input_data = ParameterString(
    name="InputData",
    default_value=input_data_path,
)
batch_data = ParameterString(
    name="BatchData",
    default_value=batch_data_path,
)

# this is the 
rmse_threshold = ParameterFloat(name="RmseThreshold", default_value=best_rmse_metric)

### Setup Docker Container for Custom Processing

Fetch latest data from the offline feature store (Past 1 week)
Transform in to JSONL Test Format


In [18]:
!mkdir -p code
!mkdir docker
!pip install sagemaker-studio-image-build

mkdir: cannot create directory ‘docker’: File exists


In [11]:
%%writefile docker/Dockerfile
FROM python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 json boto3 sagemaker time awswrangler pyathena
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

Writing docker/Dockerfile


In [14]:
# push docker container
ecr_repository = 'deepar-processing-container'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [21]:
# build container
!sm-docker build -t $ecr_repository docker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Created ECR repository sagemaker-studio
...................[Container] 2024/10/15 21:16:28.146560 Running on CodeBuild On-demand

[Container] 2024/10/15 21:16:28.146574 Waiting for agent ping
[Container] 2024/10/15 21:16:28.247523 Waiting for DOWNLOAD_SOURCE
[Container] 2024/10/15 21:16:30.072083 Phase is DOWNLOAD_SOURCE
[Container] 2024/10/15 21:16:30.112993 CODEBUILD_SRC_DIR=/codebuild/output/src1727034112/src
[Container] 2024/10/15 21:16:30.113572 YAML location is /codebuild/output/src1727034112/src/buildspec.yml
[Container] 2024/10/15 21:16:30.116997 Setting HTTP client timeout to higher timeout for S3 source
[Container] 2024/10/15 21:16:30.117137 Processing environment variables
[Container] 2024/10/15 21:16:30.161892 No runtime version selected in buildspec.
[Container] 20

In [22]:
# Create ECR repository
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
!aws ecr create-repository --repository-name $ecr_repository

/bin/bash: line 1: docker: command not found


Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='utf-8'>
BrokenPipeError: [Errno 32] Broken pipe

An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'deepar-processing-container' already exists in the registry with id '343218227212'


In [24]:
# push docker image
!sm-docker tag {ecr_repository + tag} $processing_repository_uri
!sm-docker push $processing_repository_uri

usage: sm-docker [-h] {build} ...
sm-docker: error: argument subcommand: invalid choice: 'tag' (choose from 'build')
usage: sm-docker [-h] {build} ...
sm-docker: error: argument subcommand: invalid choice: 'push' (choose from 'build')


### 1. Setup Data Pre-Processing Step

In [80]:
%%writefile code/preprocessing.py
import os
import json
from time import gmtime, strftime
import datetime
import pandas as pd

# helper function to build the target time series for each store
def build_store_timeseries(store_sales, target_col):
    unique_stores = store_sales['store_nbr'].unique()
    store_timeseries = []
    for store_nbr in unique_stores:
        # get the sales data for this store and only keep the timestep and sales number
        store_data = store_sales[store_sales['store_nbr'] == store_nbr]
        store_data = store_data[['date', target_col]]

        # convert to datetime and then to series with timestep = 1d
        store_data['date'] = pd.to_datetime(store_data['date'])
        
        store_data = store_data.set_index('date')
        store_data = store_data.resample('D').sum()
        store_ts = store_data.iloc[:, 0]

        # add to list
        store_timeseries.append(store_ts)    
    return store_timeseries

# helper function to write ts datasets to json
def write_dicts_to_file(path, data):
    with open(path, "wb") as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode("utf-8"))


if __name__ == "__main__":
    
    base_dir = "/opt/ml/processing"
    
    #load input data
    input_data_path = os.path.join("/opt/ml/processing/input", "input_data.csv")
    input_df = pd.read_csv(input_data_path)
    

Overwriting code/preprocessing.py


In [81]:
# setup script processor
from sagemaker.processing import ScriptProcessor
script_processor = ScriptProcessor(
    image_uri='343218227212.dkr.ecr.us-east-1.amazonaws.com/deepar-processing-container:latest',
    role=role,
    instance_count=1,
    instance_type='ml.m5.2xlarge',
    command=['python3'],
    base_job_name="deepar-feature-process",
    sagemaker_session=pipeline_session,
)

In [82]:
processor_args = script_processor.run(
    inputs=[
        ProcessingInput(source=input_data_file, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    code="code/preprocessing.py",
)

step_process = ProcessingStep(name="DeepARFeatureProcess", step_args=processor_args)

/opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [83]:
pipeline_name = f"DeepARTrainDeployPipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        instance_type,
        model_approval_status,
        input_data,
        batch_data,
        rmse_threshold,
    ],
    steps=[step_process],
)

In [84]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:343218227212:pipeline/DeepARTrainDeployPipeline',
 'ResponseMetadata': {'RequestId': '5b0e57ce-f547-4556-9e05-ccae370ab3b8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5b0e57ce-f547-4556-9e05-ccae370ab3b8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '93',
   'date': 'Tue, 15 Oct 2024 22:58:50 GMT'},
  'RetryAttempts': 0}}

In [85]:
execution = pipeline.start()

In [86]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:343218227212:pipeline/DeepARTrainDeployPipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:343218227212:pipeline/DeepARTrainDeployPipeline/execution/wzyrg23jznzh',
 'PipelineExecutionDisplayName': 'execution-1729033132272',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'deepartraindeploypipeline',
  'TrialName': 'wzyrg23jznzh'},
 'CreationTime': datetime.datetime(2024, 10, 15, 22, 58, 52, 216000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 10, 15, 22, 58, 52, 216000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:343218227212:user-profile/d-nzj1ohif3tlp/default-20241007T091581',
  'UserProfileName': 'default-20241007T091581',
  'DomainId': 'd-nzj1ohif3tlp',
  'IamIdentity': {'Arn': 'arn:aws:sts::343218227212:assumed-role/AmazonSageMaker-ExecutionRole-20241007T091581/SageMaker',
   'PrincipalId': 'AROAU72LGRAGGJLCV6WF3:SageMaker'}},
 

In [87]:
execution.list_steps()

[{'StepName': 'DeepARFeatureProcess',
  'StartTime': datetime.datetime(2024, 10, 15, 22, 58, 53, 378000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:343218227212:processing-job/pipelines-wzyrg23jznzh-DeepARFeatureProcess-8wvjDAAm6Y'}},
  'AttemptCount': 1}]